In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import nltk
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud,STOPWORDS
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import string
import textblob as tb
from tqdm import tqdm

### Exploring the data

In [2]:
colnames = ['num', 'comment', 'like', 'dislike']
df = pd.read_csv('naver/0701.csv', encoding='utf-8', header=None)

df.columns = colnames
print(len(df))
df.head()

26567


,num,comment,like,dislike
0,0,"""연대 법대출신 토익925점 kt취업vs건대 충주캠 무스펙 귀걸이낀 증명사진 이력서...",19,1
1,1,"""전직 공안 검사 출신 이자나...없는 일도 만들고 없던 말도 만들고...그렇게 일...",17,0
2,2,"""교활이! 이새.키는 503호뇬 밑에 있을때는 다들 벙어리인줄 알았어.....헐...",23,7
3,3,"""이렇게 입이 가벼운 작자가 어찌 대권을 노린다는건지... 한심하지 짝이 없는 개누리당""",15,1
4,4,"""연세대나온 아들이 KT간게 뭐대수라고 지잡나온 귀걸이 아빠나 조사해봐라 정권의 개들아""",21,8


In [ ]:
# # 공감비율 계산

# def favor_cmt(l, d):
#     return np.log(l+.5)/(np.log(l+d+1)+1)

# df['empathize'] = df.apply(lambda x: favor_cmt(x['like'], x['dislike']), axis=1)

# dat = df[(df['empathize'] >= 0.70) & (df['month']==7)] # 공감도 0.8 이상의 댓글만 가져옴
# len(dat) # 가져온 후 댓글의 갯수

In [3]:
# 공감 = 0, 공감 < 비공감 댓글의 수

print('comments with zero like : %d' %len(df[df['like']==0]))
print('comments with likes < dislikes : %d' %len(df[(df['dislike']-df['like'])>0]))

# 공감 = 0, 공감 < 비공감 댓글 제거

df = df[df['like']!=0]
df = df[(df['like']-df['dislike'])>0]
print('number of comments after removal : %d' %len(df))

comments with zero like : 9068
comments with likes < dislikes : 4092
number of comments after removal : 13529


In [4]:
comments = []
comments.extend(list(df.comment.values)) # 댓글만 리스트로 저장

### Data celaning

In [5]:
from string import punctuation
import re

def clean_text(text):
    cleaned_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ]', ' ', text) # 한글이 아니면 공백으로 치환
#     cleaned_text = re.sub('[0-9]', ' ', text)
#     cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"\'\·]',
#                           ' ', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text) # 여러개의 공백을 하나의 공백으로 치환

    return cleaned_text

text = [clean_text(x) for x in comments] # 한글외 모든 문자 제거
text = [x.strip() for x in text] # 문자열 양 끝에 있는 공백과 \n 기호 삭제 
text = [n for n in text if len(n) >=10] # 글자 수 가 10이상인 댓글만 사용
text = pd.DataFrame(text, columns=['cmt']) # dataframe 형식으로 변환
print(len(text))
text.head()

12702


,cmt
0,연대 법대출신 토익 점 취업 건대 충주캠 무스펙 귀걸이낀 증명사진 이력서 공기업 취...
1,전직 공안 검사 출신 이자나 없는 일도 만들고 없던 말도 만들고 그렇게 일반인 빵갱...
2,교활이 이새 키는 호뇬 밑에 있을때는 다들 벙어리인줄 알았어 헐 근데 지금보니 완전...
3,이렇게 입이 가벼운 작자가 어찌 대권을 노린다는건지 한심하지 짝이 없는 개누리당
4,연세대나온 아들이 간게 뭐대수라고 지잡나온 귀걸이 아빠나 조사해봐라 정권의 개들아


In [6]:
from konlpy.tag import Komoran

dicpath = 'user_dict.txt' 
komoran = Komoran(userdic=dicpath) # 사용자 사전 등록

stopwords = pd.read_excel('stopwords100.xlsx', encoding='utf-8') # 불용어 사전 로드
stopwords = set(list(stopwords['Column1']))

text['cmt'] = text.apply(lambda row: komoran.nouns(row['cmt']), axis=1) # Komoran 형태소 분석기에서 명사로 인식되는 단어만 사용
text['cmt'] = text['cmt'].apply(lambda x: [word for word in x if word not in stopwords]) # 불용어 제거
text.head()

C:\ProgramData\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


,cmt
0,"[연대, 법대, 출신, 토익, 취업, 건, 대, 충주, 스펙, 귀걸이, 증명, 사진..."
1,"[전직, 공안, 검사, 출신, 이자, 일도, 만들고, 말도, 만들고, 그렇게, 일반..."
2,"[교활이, 이새, 키, 호, 뇨, 밑, 있을때, 다, 벙어리, 인줄, 근데, 보니,..."
3,"[이렇게, 입, 작자, 어찌, 대권, 건지, 한심, 하지, 짝, 누리, 당]"
4,"[연세대, 아들, 간, 뭐, 대수, 라고, 귀걸이, 아빠, 조사, 해봐라, 정권]"


In [7]:
tokenized_doc = text['cmt'].apply(lambda x: [word for word in x if len(word) >1]) # 한 글자인 단어 제거
tokenized_doc.head()

0    [연대, 법대, 출신, 토익, 취업, 충주, 스펙, 귀걸이, 증명, 사진, 이력서,...
1    [전직, 공안, 검사, 출신, 이자, 일도, 만들고, 말도, 만들고, 그렇게, 일반...
2    [교활이, 이새, 있을때, 벙어리, 인줄, 근데, 보니, 완전, ㅎㅎ, 두드러기, ...
3                    [이렇게, 작자, 어찌, 대권, 건지, 한심, 하지, 누리]
4              [연세대, 아들, 대수, 라고, 귀걸이, 아빠, 조사, 해봐라, 정권]
Name: cmt, dtype: object

### Word2vec

In [8]:
from gensim.models import Word2Vec
model = Word2Vec(tokenized_doc, size=150, window=5, min_count=5, workers=4, sg=1) # 댓글의 명사를 벡터로 분산표현
# note that these are not the best parameters. you should try something else to get better.

C:\ProgramData\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [9]:
model.wv.most_similar("자한당") # 입력한 단어와 가장 가까운 10개 단어 추출

[('니들', 0.9731961488723755),
 ('친일', 0.9670661091804504),
 ('자유한국당', 0.964316725730896),
 ('친일파', 0.9640794992446899),
 ('자유당', 0.9603596329689026),
 ('수구', 0.9578806757926941),
 ('보수', 0.9556160569190979),
 ('매국노', 0.9535486102104187),
 ('니네', 0.9524215459823608),
 ('애국', 0.9471304416656494)]

### Save and load the trained model

In [10]:
import pickle
filename = 'naver_01.sav' 
pickle.dump(model, open(filename, 'wb')) # 학습한 모델 저장

In [11]:
loaded_model = pickle.load(open(filename, 'rb')) # 저장한 모델 재사용
loaded_model.wv.most_similar("자한당")

[('친일파', 0.9788994193077087),
 ('자유당', 0.9688515663146973),
 ('친일', 0.9686806201934814),
 ('자유한국당', 0.9677141904830933),
 ('니들', 0.9562883377075195),
 ('애국', 0.9520573616027832),
 ('매국노', 0.9463508725166321),
 ('극우', 0.944614589214325),
 ('세력', 0.9406320452690125),
 ('니네', 0.940369725227356)]